In [1]:
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

URL = '/home/carlosdev/Descargas/Stackoverflow/Stack Overflow 11-2010/112010 Meta Stack Overflow/posts.xml'
tree = ET.parse(URL)
root = tree.getroot()

### *Top 10 post mas vistos*

In [79]:
%time
#mapper
def mapper(root: ET) -> list:
    '''
    Función que mapea y devuelve una lista de pares id del post y cantidad de vistas

    Args:
        root (ElementTree): Elementos extraidos desde el xml

    Returns:
        list: lista de pares id_post y viewcount
    '''
    post_view = []
    #itero sobre el elemento que contiene todos los post del .xml
    for i in root:
        post_view.append([i.attrib['Id'], i.attrib['ViewCount']])
    return post_view

#Reduce
def reducer(posts: list) -> None:
    '''
    Función que muestra los primeros 10 posts mas vistos

    Args:
        posts (list): lista de pares id_post y viewcount
    '''
    #ordeno la lista 
    post_view2 = sorted(posts, key=lambda x: int(x[1]), reverse=True)
    #muestro los primeros 10
    print('ID Post - ViewCount')
    print('--'*10)
    for i in range(10):
        print(f'{post_view2[i][0]}       {post_view2[i][1]}')
        print('--'*10)
reducer(mapper(root))


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs
ID Post - ViewCount
--------------------
28625       33344
--------------------
37328       28372
--------------------
31913       26601
--------------------
9134       20536
--------------------
1777       19695
--------------------
2267       15180
--------------------
7931       12584
--------------------
61142       9918
--------------------
20420       8903
--------------------
53346       8619
--------------------


### *Top 10 palabras mas nombradas en los post por tag*

In [80]:
def mapper_tags(root: ET) -> dict:
    '''
    Función que coloca todas las palabras en un unico arreglo para cada key o tag

    Args:
        root (ET): el xml extraido

    Returns:
        dict: diccionario con todas la palabras por key o tag
    '''
    def normaliza(body: str) -> str:
        '''
        Función para limpiar el texto de caracteres especiales y espacios

        Args:
            body (str): texto para limpiar y appendear

        Returns:
            str: texto limpio
        '''
        temp = body
        temp = temp.split('<a')[0]
        temp = temp.replace('<p>', '').replace('</p>', '').replace(',', '').replace('?', '').replace('/', '')\
            .replace('--', '').replace('"','').replace('\n', '').replace('<br />', '').replace('(', '').replace(')', '')\
                .replace('-', '').replace("'", "").replace('.', '').replace('  ', ' ').replace('<em>', '').replace('<br>', '')\
                    .replace('<strong>', '').replace('<code>', '').replace('<blockquote>', '').replace(':', '').replace('<ul>', '')\
                        .replace('<li>', '').replace(';', '').replace('>', '')
        temp = temp.lower()
        return temp.strip()

    dic_posts = {}#diccionario para guardar posts por tags

    for i in root:
        try:
            #extrayendo cada tag
            temp = i.attrib['Tags'].split('>')
            for j in temp:
                j = j.replace('<', '')
                j = j.strip()
                j = j.lower()
                if j != '':
                    #aca me fijo si esta el tag le agrego el body, sino creo el tag y le agrego el body
                    if j in dic_posts.keys():
                        dic_posts[j].append(normaliza(i.attrib['Body']))
                    else:
                        dic_posts[j] = []
                        dic_posts[j].append(normaliza(i.attrib['Body']))
        except:
            pass
    
    dict_palabras = {}
    for key in dic_posts.keys():
        #recorro cada key o tag
        palabras = []
        for k in dic_posts[key]:
            #recorro la lista de palabras para cada tag y creo un dict con un key unico y todas las palabras 
            # de esa key como value
            temp = k.split()
            for j in temp:
                palabras.append(j)
        dict_palabras[key] = palabras
    return dict_palabras


In [81]:
def reduce_tags(dict_palabra: dict) -> dict:
    '''
    Función reduce recibo un diccionario con todas las palabras de cada tags y muestro
    como salida un dict con las top 10 palabras para cada tag

    Args:
        dict_palabra (dict): diccionario con una lista de palabras por tags

    Returns:
        dict: diccionario con top 10 palabras mas repetidas por tag(key)
    '''
    dict_tags = {}
    for key in dict_palabra.keys():#recorro cada key o tag
        diccionario = {}
        for i in dict_palabra[key]:
            #recorro para el tag o key cada palabra si esta la sumo y sino la agrego a un dict nuevo
            if i in diccionario.keys():
                diccionario[i] += 1
            else:
                diccionario[i] = 1
        dict_ordenado = sorted(diccionario.items(), key=lambda x: x[1], reverse=True)#ordeno el dict lo 
        top_dict = [dict_ordenado[i] for i in range(10) if len(dict_ordenado) >= 10]#extraigo el top 10 palabras
        dict_tags[key] = top_dict#lo agrego al diccionario final 
    return dict_tags

In [82]:
%time
map_tag = mapper_tags(root)
redu_tag = reduce_tags(map_tag)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [86]:
len(redu_tag)

1315

In [87]:
redu_tag['discussion']

[('the', 14049),
 ('to', 10930),
 ('a', 9140),
 ('i', 8154),
 ('of', 6349),
 ('and', 6047),
 ('is', 5433),
 ('that', 5100),
 ('it', 4479),
 ('in', 3683)]

### *Tiempo de respuesta del ranking 200-300 por score*

In [50]:
#Genero una lista con el tiempo de respuesta expresado en minutos -> salida del map
def map_score(top_score: list, answer: dict) -> list:
    '''
    Función map que genera una lista con el timepo en minutos de respuesta entre
    la pregunta y la respuesta aceptada

    Args:
        top_score (list): lista con la preguntas ordenadas por score del 200-300
        answer (dict): diccionario que contiene como key el id de respuesta y valor hora creada

    Returns:
        list: lista expresada en minutos de tiempo de respuesta entre pregunta y respuesta aceptada
    '''
    list_time = []
    for res, hora_q, score in top_score:
        hora_a = answer[res]#extraigo la hora del dicc de respuestas
        diff = hora_a-hora_q
        diff_s = diff.total_seconds()
        minutos = divmod(diff_s, 60)[0]#diferencia en minutos
        list_time.append(int(minutos))#apendeando los minutos
    return list_time

In [60]:
#Genero un solo resultado de tiempo de respuesta en minutos -> salida del reduce
def reduce_score(list_time: list) -> None:
    '''
    Función reduce que muestra el resultado promedio de la lista generada anteriormente
    expresada en minutos de la diferencia entre la pregunta y respueta aceptada.

    Args:
        list_time (list): lista expresada en minutos del tiempo de respuesta
    '''
    calc = 0
    for i in list_time:
        calc += i
    print(f'Horas promedio de respuesta: {int(calc / 60 /100)} horas')
    print(f'En dias seria: {int(calc / 60 /100 /24)} dias')

In [61]:
%time
#creo una lista para las question y un diccionario para las respuestas
question = []#lista con todas las preguntas
answer = {}#diccionario con el id de respuesta como keys y la hora creada como valor
for i in root:
    temp = int(i.attrib['PostTypeId'])
    if temp == 1:#pregunto si es question
        try:#este try va porque aveces no tiene respuesta aceptada o le falta otro atributo
            if i.attrib['AcceptedAnswerId'] != '' and i.attrib['CreationDate'] != '' and i.attrib['Score'] != '':
                q_temp = []
                q_temp.append(i.attrib['AcceptedAnswerId'])
                horaQuestion = i.attrib['CreationDate'].replace('T', ' ').split('.')[0]
                horaQuestion = datetime.strptime(horaQuestion, '%Y-%m-%d %H:%M:%S')
                q_temp.append(horaQuestion)
                q_temp.append(i.attrib['Score'])
                question.append(q_temp)#lista con respuesta aceptada, hora y score
        except:
            pass
        
    else:#entro aca si es answer
        horaAnswer = i.attrib['CreationDate'].replace('T', ' ').split('.')[0]
        horaAnswer = datetime.strptime(horaAnswer, '%Y-%m-%d %H:%M:%S')
        answer[i.attrib['Id']] = horaAnswer#diccionario con key id respuesta y valor hora de creación
#genero la lista de top score de 200-300
top_score = []
question_ord = sorted(question, key=lambda x: x[2], reverse=True)
for i in range(199, 299):
    top_score.append(question_ord[i])
#Llamando a reduce_score y map_score
reduce_score(map_score(top_score, answer))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
Horas promedio de respuesta: 171 horas
En dias seria: 7 dias
